Vector stores and retrievers

Documents


LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

page_content: a string representing the content;
metadata: a dict containing arbitrary metadata. The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

In [1]:
from langchain_core.documents import Document

documents = [

Document(
page_content="Dogs are great companions, known for their loyalty and friendliness.", 
metadata={"source": "sammal-pets-doc"},),


Document(
page_content="Cats are independent pets that often enjoy their own space.",
metadata={"source": "mammal-pets-doc"},),

Document(
page_content="Goldfish are popular pets for beginners, requiring relatively simple care.", 
metadata={"source": "fish-pets-doc"},),

Document(
page_content="Parrots are intelligent birds capable of mimicking human speech.", 
metadata={"source": "bird-pets-doc"},),

Document(
page_content="Rabbits are social animals that need plenty of space to hop around.", 
metadata={"source": "mammal-pets-doc"},),
]

In [2]:
documents

[Document(metadata={'source': 'sammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
llm=ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E639EF6CD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E63A0F3D10>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

: 

In [ ]:
#VectorStores
from langchain_chroma import Chroma
vectorstore=Chroma.from_documents(documents,embedding=embeddings)
vectorstore

In [ ]:
vectorstore.similarity_search("cat")

In [ ]:
#async query
await vectorstore.asimilarity_search("cat")

In [ ]:
vectorstore.similarity_search_with_score("cat")

Retrievers


LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g. synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity search method:

In [ ]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

Vectorstores implement an as retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search type and search kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [ ]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retriever.batch(["cat","dog"])

In [ ]:
#RAG
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message="""
answer this question using the provided context only.
{question}
Context:
{context}
"""
prompt=ChatPromptTemplate.from_messages([("human",message)])

rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm
response=rag_chain.invoke("tell me about dogs")
print(response.content)

NameError: name 'retriever' is not defined